# Example: Suggested exercise for parameter estimation.

## Introduction

This example addresses the adjustment of models for the experimental data presented in Table 1. It is necessary to choose between two models to evaluate which one best fits this data:

$y = \left(\frac{α_1\cdot x}{1+β_1\cdot x}\right)$          (1)

$y = \left(α_2\cdot x^{β_2}\right)$                           (2)

on what $α_1$, $α_2$, $β_1$ e $β_2$ are the parameters to be estimated for each model. This example is proposed by Schwaab e Pinto [1].


**Table 1: Experimental data. Font: [1]**


Variables|  1 |  2 |  3 |  4 |  5  |  6  |  7  |  8  |  9  |  10 |
:------:|:--:|:--:|:--:|:--:|:---:|:---:|:---:|:---:|:---:|:---:|
$x^e$   |1.00|2.00|3.00|5.00|10.00|15.00|20.00|30.00|40.00|50.00|
$y^e$   |1.66|6.07|7.55|9.72|15.24|18.79|19.33|22.38|24.27|25.51|

The optimization problem to be solved uses the objective function of least squares weighted by the inverse of the variance, according to [2]:

$F_{Obj} = \sum_{i=1}^{NE} \left({\frac{(y^{e}_i-y_i^{m})^2}{σ^2_{i}}}\right)$ (3)

subject to (1) or (2).

To solve this problem in the MT_PEU the followed symbols will be used:

* Symbols of the independent quantities: $x_1$, $x_2$
* Symbols of the dependent quantities: $y_1$, $y_1$
* Symbols of the parameters: $α_1$, $α_2$, $β_1$, $β_2$

Sub-index 1 refers to model (1), and sub-index 2 refers to model (2).

## Packages importing

Importing libraries (packages) needed to run the code.

* **MT_PEU**: library that contains the main functionalities of the tool

    * Import the class **EstimacaoNaoLinear**, that will be used in this non-linear estimation example.

* **casadi**: library for symbolic computation

    * only the function **vertcat** will be required to build the model

In [1]:
from MT_PEU import EstimacaoNaoLinear
from casadi import vertcat # function to concatenate model expressions

## Model creation

The models (1) and (2) represent the behavior of the dependent quantity, in which the parameters $α_1$, $α_2$, $β_1$ e $β_2$ will be estimated..

These models are then defined in the form of a python subroutine (**def**) and represented by:

In [2]:
def Model(param,x,args):
    
    a1, b1, a2, b2 = param[0], param[1], param[2], param[3]
    x1, x2 = x[:,0], x[:,1]

    return vertcat(a1*x1/(1+b1*x1), a2*(x2**b2))

## Class initialization

The first step to perform the estimation is to configure the class **EstimacaoNaoLinear** through the inclusion of basic information.:

* The model,
* List of symbols of the independent quantities *(x)*; 
* List of symbols of the dependent quantities *(y)*; 
* List of symbols of the parameters *(param)*;
* List of symbols for parameters written in LaTex *(Label_latex_param)*;
* List of units of measure for independent variables *(Units_y)*;
* The project name, the folder's name where the results will be saved.

In [3]:
Estime = EstimacaoNaoLinear(Model,symbols_x=['x1','x2'],symbols_y=['y1','y2'],symbols_param=['alpha1','alpha2', 'beta1', 'beta2'],
                          label_latex_param=[r'$\alpha_1$',r'$\alpha_2$',r'$\beta_1$',r'$\beta_2$'],units_y=['kg','kg'],Folder='Exemple4')

## Data inclusion

The experimental data provided in Table 1 of the dependent quantities ($y_1$ e $y_2$) e independent quantities ($x_1$ e $x_2$), are presented below in the form of lists:

In [4]:
x1 = [1.,2.,3.,5.,10,15.,20.,30.,40.,50.]
y1 = [1.66,6.07,7.55,9.72,15.24,18.79,19.33,22.38,24.27,25.51]
x2 = [1.,2.,3.,5.,10,15.,20.,30.,40.,50.]
y2 = [1.66,6.07,7.55,9.72,15.24,18.79,19.33,22.38,24.27,25.51]

The MT_PEU **needs** the **uncertainties of the experimental data** (ux1, ux2, uy1, uy2) to be informed. In this example, the value 1 has been assumed for all uncertainties.

In [5]:
ux1 = [1]*10
ux2 = [1]*10
uy1 = [1]*10
uy2 = [1]*10

**Entering the experimental data in MT_PEU:**

The *setDados* method is used to include the data for dependent and independent quantities. Syntax:

* Quantity identification, whether it is independent or dependent: 0 or 1 (respectively)
* The experimental data and their uncertainties must be entered in sequence in the form of tuples.

In [6]:
Estime.setDados(0,(x1,ux1),(x2,ux2))
Estime.setDados(1, (y1, uy1), (y2, uy2))

The setConjunto method defines that the experimental data previously entered will be used as a data set for which parameters will be estimated. The degrees of freedom for the quantity $ x $ and $ y $ are defined in *glx* and *gly*, respectively:

In [7]:
Estime.setConjunto(dataType='estimacao',glx=[], gly=[])

## Optimization

In this example, the user can choose the algorithm to be used in the optimization. 
Available: 'ipopt', 'bonmin' and 'sqpmethod'.

If the user does not choose any optimization method, the default algorithm will be used: ipopt, with initial estimative equal to [3, 0.1, 5,0.4].

**Lower_bound** and **upper_bound:** They refer to the upper and lower limits of the parameters related to their physical characteristics;

**algorithm:** Informs the optimization algorithm that will be used. Each algorithm has its own keywords;

**optimizationReport:** Informs whether the optimization report should be created (True or False);

**parametersReport:** Informs whether the parameters report should be created (True or False).

In [8]:
Estime.optimize(initial_estimative=[3,0.1,5,0.4], algorithm='ipopt', lower_bound=[0.2,0.09,3.1,0.3], upper_bound=[3.6,0.3,5.6,0.6],
                optimizationReport = True, parametersReport = False)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************



## Parameters uncertainty


In this example, it is possible to choose the method used to evaluate uncertainty. 
Available: 2InvHessiana, Geral, SensibilidadeModelo. 
By definition the likelihood region filling is 'True', if necessary this option can be changed.

**uncertaintyMethod:** method for calculating the covariance matrix of the parameters.

**parametersReport:** Informs whether the parameters report should be created (True or False).

**objectiveFunctionMapping:** Deals with mapping the objective function (True or False).

**lower_bound:** Lower limit of parameters for mapping the coverage region.

**upper_bound:** Upper limit of parameters for mapping the coverage region.

If the user **does not** have an interest in **evaluating the uncertainty of the parameters**, just **do not** perform **Estime.parametersUncertainty**.

In [ ]:
Estime.parametersUncertainty(uncertaintyMethod='2InvHessiana', objectiveFunctionMapping=True, lower_bound=[1,0.04,1.75,0.175], upper_bound=[4.5,0.16,6.75,1],
                             parametersReport = True)

## Prediction and residual analysis

The prediction method evaluates the dependent quantity based on the estimated parameters. To give more credibility to the results, the covariance matrix of the estimates obtained through the model is also evaluated. 
 
The residue analysis provides information concerning the quality of the model, for example: (i) trend relationship not included in the model, (ii) undue correlations, etc. Such assessments are based on statistical tests such as homoscedasticity and $ðchi^2$.

The prediction and the residual analysis are preferably performed with validation data when available.

**export_y:** Exports the calculated data of y, its uncertainty, and degrees of freedom in a txt with comma separation (True or False);

**export_y_xls:** Exports the calculated data of y, its uncertainty, and degrees of freedom in a xls (True or False);

**export_cov_y:** Exports the covariance matrix of y (True or False);

**export_x:** Exports the calculated data of x, its uncertainty, and degrees of freedom in a txt with comma separation(True or False);

**export_cov_x:** Exports the covariance matrix of x (True or False).

In [ ]:
Estime.prediction(export_y=True,export_y_xls=True, export_cov_y=True, export_x=True, export_cov_x=True)
Estime.residualAnalysis(report=True)

## Plots and reports

At this stage, the results obtained by the program are exported: reports and graphs. 
The graphs are generated according to the steps that have been performed. In the reports, information about the statistical tests, objective function value, covariance matrix of the parameters, optimization status, among others, are printed.

In [ ]:
Estime.plots()
Estime.reports()

## References: 

[1] SRINIVASAN, R.; LEVI, A. A.. Kinetics of the Thermal Isomerization of Bicyclo [2.1.1 ]hexane. Journal Of The American Chemical Society, [s.l.], v. 85, n. 21, p.3363-3365, 5 nov. 1963. American Chemical Society (ACS)

[2] SCHWAAB, M.M.;PINTO, J.C. Análise de Dados Experimentais I: Fundamentos da Estátistica e Estimação de Parâmetros. 
Rio de Janeiro: e-papers, 2007.

INMETRO.: Avaliação de dados de medição — Guia para a expressão de incerteza de medição. Rio de Janeiro: Jcgm, 2008.